In [1]:
using Revise
push!(LOAD_PATH,"/scratch/Codes/RandomMeas_dev.jl/src/")
#push!(LOAD_PATH,"/scratch/Codes/RandomMeas_dev.jl/")
#import Pkg; Pkg.activate("../")
using ProgressBars
using Random
using ITensors
using RandomMeas
using StatsBase
using TimerOutputs

[ Info: Precompiling RandomMeas [top-level]


In [311]:
#Step 1: Calibration Data acquisition
N = 2
ξ = siteinds("S=1/2", N;addtags="output")
s = siteinds("S=1/2", N;addtags="input")
v = siteinds("S=1/2", N;addtags="virtual")
χ = 2
NM = 1000
nu = 100
states = ["Dn" for n in 1:N]
ψ = MPS(ξ,states)
ρ = state_to_dm(ψ)
u = [get_rotations(ξ,1) for r in 1:nu]
p = 0.8
Λ =  get_random_Pauli_noise(ξ,s,p)
M = AcquireMeasurementChannel(ρ,v,u,NM;Λ=Λ);
M_shift = AcquireMeasurementChannel(ρ,v,u);
M0 = IdealMeasurementChannel(ρ,v,1/3);
MC = M-M_shift+M0;

In [312]:
function loss3(c::Vector{ITensor},M::ITensor,ρ::MPO)
    M_model = MeasurementChannel(c,ρ)
    X = M-M_model
    return real(inner(X,X))
end
loss4(x) = loss3(x,MC,ρ)

loss4 (generic function with 1 method)

In [313]:
#Step 2: Find c to model the measurement channel
using Zygote
using OptimKit
optimizer = LBFGS(; maxiter=25, verbosity=1)
function loss_and_grad(x)
  y, (∇,) = withgradient(loss4, x)
  return y, ∇
end
c0 = randomMPS(Float64,v,;linkdims=χ).data
c, fs, gs, niter, normgradhistory = optimize(loss_and_grad, c0, optimizer);
#@show c[1], c[2]
#@show c[1]*c[2]

┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point without satisfying Wolfe conditions?
└ @ OptimKit ~/.julia/packages/OptimKit/xpmbV/src/linesearches.jl:189
┌ Warning: Linesearch bracket converged to a point w

In [314]:
#Step 3 Build the channel Mc for any input state
ρempty = MPO(ξ)
for i in 1:N
    ρempty[i] = δ(ξ[i],s[i])*δ(ξ[i]',s[i]')
end
Mc = MeasurementChannel(c,ρempty);
d0 = randomMPS(Float64,v,;linkdims=χ).data;

In [315]:
function loss5(d::Vector{ITensor})
    MdMc = MeasurementChannel(d,Mc)
    Id = 1.
    for i in 1:N
        Id *= δ(ξ[i],s[i])*δ(ξ[i]',s[i]')
    end
    X = MdMc -Id
    return inner(X,X)
end

loss5 (generic function with 1 method)

In [316]:
#Step 4: Find $d$ to form the inverse channel
using Zygote
using OptimKit
optimizer = LBFGS(; maxiter=25, verbosity=1)
function loss_and_grad(x)
  y, (∇,) = withgradient(loss5, x)
  return y, ∇
end
d, fs, gs, niter, normgradhistory = optimize(loss_and_grad, d0, optimizer);
#println(" --- Inversion Channel --- ")
#@show d[1], d[2], d[1]*d[2]

[ Info: LBFGS: converged after 19 iterations: f = 0.000000000000, ‖∇f‖ = 7.1772e-09


In [317]:
#Application to a noisy random MPS state
ψ  = randomMPS(ComplexF64,ξ;linkdims=χ)
ψ = MPS(ξ,states)
ps = 0.8
σ = state_to_dm(ψ)
ρ = copy(σ)
Λs = get_random_Pauli_noise(ξ,s,ps)
Apply_channel!(Λs,ρ)
#get measurements
M = AcquireMeasurementChannel(ρ,v,u,NM;Λ=Λ);
ρe = MeasurementChannel(d,M)
#CRM shift
σ_shift = IdealMeasurementChannel(AcquireMeasurementChannel(σ,v,u),v,3.);
ρe = ρe-σ_shift+flatten(σ)

ρe_nonmit = IdealMeasurementChannel(M,v,3.)-σ_shift+flatten(σ);

In [318]:
println("Exact fidelity ", inner(σ,ρ))
println("Estimated fidelity ",trace(multiply(flatten(σ),ρe),ξ))
println("Estimated fidelity (no readout correction) ",trace(multiply(flatten(σ),ρe_nonmit),ξ))

Exact fidelity 0.844611197953546 + 0.0im
Estimated fidelity 0.864062091911785 + 0.0im
Estimated fidelity (no readout correction) 0.7135561146399719 + 0.0im
